In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
distanse_sensor = 10e-6 # not used

# using a pixel based method to generating the mask is slow.
# What is does is:
# - check each pixel and create a box if the pixel is opaque.
# - merge all the boxes together (if simplify is True)
# Thus the mask is downsampled to a size where the processing time is reasonable. 
downsample = 16 

# simplifying the model into one solid object takes a considerable amount of time. But makes post processing easier and the resulting file much smaller
simplify = True

In [ ]:
from lensless.hardware.mask import FresnelZoneAperture, CodedAperture
from lensless.hardware.mask_3dmodel import Mask3DModel, SimpleFrame, CodedAppertureConnection, ThreePointConnection, HemnisphereLenses, mold_from_mask
import cadquery as cq
from datetime import datetime

def fza_gen(r, h=0.5):
    mask = FresnelZoneAperture.from_sensor(
        sensor_name="rpi_hq", downsample=downsample, distance_sensor=distanse_sensor, radius=r
    )

    mask_model = Mask3DModel.from_mask(mask, 
        frame = SimpleFrame(size=(10, 13)), 
        connection = ThreePointConnection(line_width=0.2, mask_radius=r*1e3), 
        height=h,
        simplify=simplify,
        generate = True,
        show_axis=False,
    )

    date_str = datetime.now().strftime("%d.%m.%Y-%H.%M")
    mask_model.save(f"out/fza_downsample{downsample}_radius{r*1000:.2f}mm{'_min' if simplify else ''}_{date_str}.step")
    return mask_model

# coded aperture
def ca_gen(n=4, r=0.03, h=0.5):
    n_bits=n
    joint_radius = r
    simplify = True # simplifying the model into one solid object takes a considerable amount of time. But makes post processing easier.

    mask = CodedAperture.from_sensor(
        sensor_name="rpi_hq", downsample=downsample, distance_sensor=distanse_sensor, n_bits=n_bits
    )

    mask_model = Mask3DModel.from_mask(mask, 
        frame = SimpleFrame(size=(10, 13)), 
        connection = CodedAppertureConnection(joint_radius=joint_radius),
        height=h,
        simplify=simplify,
        generate = True,
        show_axis=False,
    )

    date_str = datetime.now().strftime("%d.%m.%Y-%H.%M")
    mask_model.save(f"out/coded_aperture_downsample{downsample}_nbits{n_bits}_r{joint_radius}{'_min' if simplify else ''}_{date_str}.step")
    return mask_model

def mold_gen(N=24, dmin=0.5e-3, dmax=1.5e-3):
    mask_model = HemnisphereLenses(N=N, dmin=dmin, dmax=dmax)

    model = mold_from_mask(mask_model.model, size=(20e-3, 23e-3, 3e-3))

    fname = f'out/Hemnisphere_mold_dmin_{dmin}_dmax_{dmax}_N_{N}.step'

    cq.exporters.export(model, fname)

    return model

In [ ]:
# fza_gen(1e-3, 0.5)
# fza_gen(1e-3, 1)
# fza_gen(1.3e-3, 0.5)
# fza_gen(1.3e-3, 1.0)
# fza_gen(1.0e-3, 0.5)
# fza_gen(1.0e-3, 1.0)
# # display(model.model)
# ca_gen(n=3, r=0.06, h=0.5)
# ca_gen(n=3, r=0.06, h=1)
# ca_gen(n=4, r=0.06, h=0.5)
# ca_gen(n=4, r=0.06, h=1)

# mold_gen(N=30, dmin=0.1e-3, dmax=1.5e-3)
# mold_gen(N=30, dmin=0.1e-3, dmax=1e-3)
# mold_gen(N=30, dmin=0.5e-3, dmax=1.5e-3)
model = mold_gen(N=10, dmin=0.1e-3, dmax=1.5e-3)

# this renders the model in the notebook interactively. This is very nice to visualize the model
display(model)